In [1]:
from transformers import pipeline
import pandas as pd
import newsRev_DeBERTa
import newsRev_BART



DeBERTa_Function(ab)
0:00:00.000097
Robotics
Relevant
0:00:03.301612


In [2]:
from langchain import PromptTemplate, LLMChain, OpenAI 
from langchain.memory import ConversationBufferMemory

import os

In [3]:
api_key = os.getenv("OPENAI_API_KEY")


In [4]:
df = pd.read_csv('News_Relevance_LLM/news relevancy - api_data.csv')

In [5]:
df.head()

,Title,Link,Article,newsCategory,newsRelevancy,groundTruth,Unnamed: 6
0,Apple iOS 17.4: iMessage Gets Post-Quantum Enc...,https://www.wired.com/story/apple-pq3-post-qua...,Apple is launching its first post-quantum prot...,Communications Technologies,Relevant,Relevant,NaN
1,These Companies Have a Plan to Kill Apps,https://www.wired.com/story/these-companies-ha...,"In another example, Yue asks the phone to find...",AI,Relevant,Relevant,NaN
2,Inside Finland’s state-of-the-art quantum comp...,https://thenextweb.com/news/finland-quantum-co...,"In 1965, cryogenics pioneer Olli V. Lounasmaa ...",Quantum Computing,Relevant,Relevant,NaN
3,New AI and 5G advancements will usher in the e...,https://www.businessinsider.com/ai-edge-comput...,The AI boom is breathing life into edge comput...,AI,Relevant,Relevant,NaN
4,Seeking nominations for the top creative agenc...,https://www.businessinsider.com/seeking-nomina...,Business Insider is seeking nominations for a ...,AI,Relevant,Relevant,NaN


In [6]:
def relevance_GPT(article_insert):
    print('RELEVANCE GPT FUNCTION WORKING')
    try:
        #Defining Function + ChatGPT
        #Langchain implementation
        template = """ You are a bot that will be given an article and to deem if it is relevant to technology. 
        Please answer with Relevant or Irrelevant 

        Technology also includes the following:
        
        AI includes Discriminative AI, Machine Learning, Generative AI

        Quantum Computing includes Quantum Internet, Quantum Communications, Quatum Computing

        Green Computing includes Green Serverless Computing, Green Edge Applications, Green Data Streaming

        Robotics

        Trust Technologies includes Privacy Enhancing Technologies, Regulation Technologies, Al Governance Technologies

        Anti-disinformation technologies includes Content Provenance Technologies, Anti-misinformation technologies, Detection of Generated Al content

        Communications Technologies includes 5G, Networks, Seamless

        In terms of the article information, classify them as Relevant or Irrelevant. 

        Make sure that there are no spacing before the first word.


        Human: {article}
        Assistant:"""

        prompt = PromptTemplate(
            input_variables=["article"], 
            template=template
        )

        chatgpt_chain = LLMChain(
            llm = OpenAI(openai_api_key=api_key,model="gpt-3.5-turbo-instruct", temperature=0), 
            prompt=prompt, 
            verbose=False,  # Set verbose to False to suppress output
            memory=ConversationBufferMemory(memory_key="history", input_key="article")
        )
        #Let ChatGPT to Categorise
        output = chatgpt_chain.predict(article=article_insert) 

        #To make sure that there are no spacing which ChatGPT outputs " Category_Name" -> "Category_Name"
        output = output.strip()

        return output

    except Exception as e:
        print(f"An error occurred: {e}")
        return "Irrelevant"  # Return 'Irrelevant'' category in case of error



In [9]:
def newsRelevancy(article_content):
    countRelevance = 0
    
    gpt_Relevance = relevance_GPT(article_content)
    if (gpt_Relevance == "Relevant"):
        countRelevance += 1

    #print("GPT:")
    #print(gpt_Relevance)

    #LLM 1
    bart_Relevance = newsRev_BART.bart_Function(article_content)
    if (bart_Relevance == "Relevant"):
        countRelevance += 1
    #print("BART:")
    #print(bart_Relevance)
    #LLM 2
    deberta_relevance = newsRev_DeBERTa.DeBERTa_Function(article_content)
    if (deberta_relevance == "Relevant"):
        countRelevance += 1
    #print("deBerta:")
    #print(deberta_relevance)
        
    #False to deem article not relevant
    if countRelevance >= 2:
        #print("Final: Relevant")
        return 'Relevant'
    else:
        #print("Final: Irrelevant")
        return 'Irrelevant'


In [10]:
print(newsRelevancy("Finance Business"))

RELEVANCE GPT FUNCTION WORKING


/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


GPT:
Irrelevant
Finance
BART:
Irrelevant
Business
deBerta:
Irrelevant
Final: Irrelevant
Irrelevant


In [11]:
df['Voting'] = df['Article'].apply(newsRelevancy)

RELEVANCE GPT FUNCTION WORKING
GPT:
Relevant
Quantum Computing
BART:
Relevant
Quantum Computing
deBerta:
Relevant
Final: Relevant
RELEVANCE GPT FUNCTION WORKING
GPT:
Relevant
AI
BART:
Relevant
AI
deBerta:
Relevant
Final: Relevant
RELEVANCE GPT FUNCTION WORKING
GPT:
Relevant
Quantum Computing
BART:
Relevant
AI
deBerta:
Relevant
Final: Relevant
RELEVANCE GPT FUNCTION WORKING
GPT:
Relevant
General
BART:
Irrelevant
AI
deBerta:
Relevant
Final: Relevant
RELEVANCE GPT FUNCTION WORKING
GPT:
Relevant
Business
BART:
Irrelevant
AI
deBerta:
Relevant
Final: Relevant
RELEVANCE GPT FUNCTION WORKING
GPT:
Relevant
General
BART:
Irrelevant
General
deBerta:
Irrelevant
Final: Irrelevant
RELEVANCE GPT FUNCTION WORKING
GPT:
Relevant
AI
BART:
Relevant
AI
deBerta:
Relevant
Final: Relevant
RELEVANCE GPT FUNCTION WORKING
GPT:
Relevant
Finance
BART:
Irrelevant
General
deBerta:
Irrelevant
Final: Irrelevant
RELEVANCE GPT FUNCTION WORKING
GPT:
Relevant
AI
BART:
Relevant
AI
deBerta:
Relevant
Final: Relevant
RELEVANC

In [12]:
df['Voting']

0        Relevant
1        Relevant
2        Relevant
3        Relevant
4        Relevant
          ...    
145    Irrelevant
146    Irrelevant
147    Irrelevant
148    Irrelevant
149    Irrelevant
Name: Voting, Length: 150, dtype: object

In [13]:

# Separate predictions and ground truth
predictions = df['Voting']  # Assuming predictions are in the first row
ground_truth = df['groundTruth']  # Assuming ground truth is in the second row



In [14]:

#print(predictions)

# Compare predictions and ground truth
correct_predictions = (predictions == ground_truth).sum()
total_predictions = len(predictions)

# Print results
print("Predictions:", predictions.values)
print("Ground Truth:", ground_truth.values)
print("Number of correct predictions:", correct_predictions)
print("Total predictions made:", total_predictions)
print("Accuracy:", correct_predictions / total_predictions)

Predictions: ['Relevant' 'Relevant' 'Relevant' 'Relevant' 'Relevant' 'Irrelevant'
 'Relevant' 'Irrelevant' 'Relevant' 'Irrelevant' 'Irrelevant' 'Relevant'
 'Relevant' 'Relevant' 'Relevant' 'Irrelevant' 'Relevant' 'Relevant'
 'Relevant' 'Relevant' 'Relevant' 'Relevant' 'Relevant' 'Irrelevant'
 'Relevant' 'Relevant' 'Irrelevant' 'Relevant' 'Relevant' 'Relevant'
 'Irrelevant' 'Irrelevant' 'Relevant' 'Relevant' 'Relevant' 'Irrelevant'
 'Relevant' 'Irrelevant' 'Relevant' 'Relevant' 'Relevant' 'Relevant'
 'Relevant' 'Relevant' 'Relevant' 'Irrelevant' 'Relevant' 'Relevant'
 'Irrelevant' 'Relevant' 'Relevant' 'Irrelevant' 'Relevant' 'Relevant'
 'Relevant' 'Relevant' 'Irrelevant' 'Relevant' 'Relevant' 'Relevant'
 'Relevant' 'Relevant' 'Relevant' 'Relevant' 'Relevant' 'Irrelevant'
 'Relevant' 'Relevant' 'Irrelevant' 'Relevant' 'Relevant' 'Irrelevant'
 'Irrelevant' 'Relevant' 'Relevant' 'Irrelevant' 'Relevant' 'Irrelevant'
 'Relevant' 'Irrelevant' 'Irrelevant' 'Relevant' 'Irrelevant' 'Relevant'
 

In [15]:
#As it took pretty long to run the Voting. I will export out the column.
#column_to_export = df['Voting']

# Export the column to a CSV file
#column_to_export.to_csv('voting_column.csv', index=False)